In [23]:
using Oceananigans
using Oceananigans.OutputWriters, Oceananigans.Fields
using Oceananigans.Utils:day, hour, minute, second

In [24]:
#grid1 = RegularCartesianGrid(size=(200, 100, 100), y=(-2500*10^3,2500*10^3), x=(-7000*10^3,7000*10^3), z=(-500,0),topology = (Bounded, Bounded, Bounded))
grid1 = RegularCartesianGrid(size=(50, 25, 40), y=(-2500*10^3,2500*10^3), x=(-7000*10^3,7000*10^3), z=(-350,0),topology = (Bounded, Bounded, Bounded))

RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}
                   domain: x ∈ [-7.0e6, 7.0e6], y ∈ [-2.5e6, 2.5e6], z ∈ [-350.0, 0.0]
                 topology: (Bounded, Bounded, Bounded)
  resolution (Nx, Ny, Nz): (50, 25, 40)
   halo size (Hx, Hy, Hz): (1, 1, 1)
grid spacing (Δx, Δy, Δz): (280000.0, 200000.0, 8.75)

In [25]:
@inline Usurface_wind_flux(x, y, t, u) = 1.3*1.1e-3*(((x*10^(-6)+12.0)/(19.0))*(cos(y/(2.0*6371.0e3))*(1.0+12.0tanh(sin(2*pi*t/17280000)+0.8)+0.125*randn())+u))^2/1025
@inline Vsurface_wind_flux(x, y, t, v) = 1.3*1.1e-3*(sin(y/6371e3)*((5.0+2tanh(sin(2*pi*t/17280000)+0.6)+0.125*randn())))^2/1025
u_wind_stress_bc = FluxBoundaryCondition(Usurface_wind_flux,field_dependencies=:u)
v_wind_stress_bc = FluxBoundaryCondition(Vsurface_wind_flux,field_dependencies=:v)
noslip=ValueBoundaryCondition(0.0)
u_bcs = UVelocityBoundaryConditions(grid1, top=u_wind_stress_bc, bottom=noslip, north=noslip,south=noslip)
v_bcs = VVelocityBoundaryConditions(grid1, bottom=noslip, top=v_wind_stress_bc,west=noslip,east=noslip)
w_bcs = WVelocityBoundaryConditions(grid1,east=noslip,west=noslip,north=noslip,south=noslip)

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Value,Float64}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{NormalFlow,Nothing},BoundaryCondition{NormalFlow,Nothing}}

In [26]:
@inline tflux(x,y,t,T)=-(185.0-(22.5)*(T-24.))/(4000.0*1025.0)
t_bcs= TracerBoundaryConditions(grid1, bottom = ValueBoundaryCondition(4.0), top = FluxBoundaryCondition(tflux,field_dependencies=:T))

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Flux,Oceananigans.BoundaryConditions.ContinuousBoundaryFunction{Nothing,Nothing,Nothing,Nothing,typeof(tflux),Nothing,Tuple{Symbol},Nothing,Nothing}}}

In [27]:
s_bcs=TracerBoundaryConditions(grid1,bottom=ValueBoundaryCondition(34.5))

Oceananigans.FieldBoundaryConditions (NamedTuple{(:x, :y, :z)}), with boundary conditions
├── x: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
├── y: CoordinateBoundaryConditions{BoundaryCondition{Flux,Nothing},BoundaryCondition{Flux,Nothing}}
└── z: CoordinateBoundaryConditions{BoundaryCondition{Value,Float64},BoundaryCondition{Flux,Nothing}}

In [28]:
model1=IncompressibleModel(grid=grid1,architecture = CPU(),float_type=Float64,clock = Clock(time=0.0),coriolis = BetaPlane(rotation_rate=7.292115e-5, latitude=0, radius=6371e3),
tracers=(:T, :S),buoyancy=SeawaterBuoyancy(),closure=AnisotropicDiffusivity(νh=1e2, νz=1e-2, κh=1e2, κz=1e-5),timestepper=:RungeKutta3
,boundary_conditions=(u=u_bcs,v=v_bcs,w=w_bcs,T=t_bcs,S=s_bcs))

IncompressibleModel{CPU, Float64}(time = 0 seconds, iteration = 0) 
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=50, Ny=25, Nz=40)
├── tracers: (:T, :S)
├── closure: AnisotropicDiffusivity{Float64,Float64,Float64,NamedTuple{(:T, :S),Tuple{Float64,Float64}},NamedTuple{(:T, :S),Tuple{Float64,Float64}},NamedTuple{(:T, :S),Tuple{Float64,Float64}}}
├── buoyancy: SeawaterBuoyancy{Float64,LinearEquationOfState{Float64},Nothing,Nothing}
└── coriolis: BetaPlane{Float64}

In [29]:
println(model1.velocities)
println(model1.tracers)

(u = Field located at (Face, Cell, Cell)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (53, 27, 42)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=50, Ny=25, Nz=40)
└── boundary conditions: x=(west=NormalFlow, east=NormalFlow), y=(south=Value, north=Value), z=(bottom=Value, top=Flux), v = Field located at (Cell, Face, Cell)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (52, 28, 42)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=50, Ny=25, Nz=40)
└── boundary conditions: x=(west=Value, east=Value), y=(south=NormalFlow, north=NormalFlow), z=(bottom=Value, top=Flux), w = Field located at (Cell, Cell, Face)
├── data: OffsetArrays.OffsetArray{Float64,3,Array{Float64,3}}, size: (52, 27, 43)
├── grid: RegularCartesianGrid{Float64, Bounded, Bounded, Bounded}(Nx=50, Ny=25, Nz=40)
└── boundary conditions: x=(west=Value, east=Value), y=(south=Value, north=Value), z=(bottom=NormalFlow, top=NormalFlow))
(T

In [30]:
@inline thermoc(x, y, z) = 16+11*tanh((z+70)/20)+1*randn()
@inline sal(x, y, z) = 35-0.5*tanh((z+70)/20)+0.5*randn()
@inline u0(x,y,z)=-0.1*tanh((z+70)/10)+0.05*randn()
@inline vel0(x,y,z)=0.05randn()
set!(model1,T=thermoc,S=sal,u=u0,w=vel0,v=vel0)

In [31]:
wizard = TimeStepWizard(cfl=0.4, Δt=0.1, max_Δt=14400.)

TimeStepWizard{Float64}(0.4, Inf, 2.0, 0.5, 14400.0, 0.0, 0.1)

In [32]:
simulation = Simulation(model1, Δt=1., stop_time=200day, iteration_interval=20)

fields = Dict("u" => model1.velocities.u,"v" => model1.velocities.v,"w" => model1.velocities.w, "T" => model1.tracers.T, "S"=>model1.tracers.S)

# Note that model.velocities is NamedTuple
simulation.output_writers[:fields] = JLD2OutputWriter(model1, fields,
                                                          prefix = "EquatorialβData",
                                                          schedule = TimeInterval(12hour))

JLD2OutputWriter scheduled on TimeInterval(12 hours):
├── filepath: ./EquatorialβData.jld2
├── 5 outputs: ["v", "S", "w", "T", "u"]
├── field slicer: FieldSlicer(:, :, :, with_halos=false)
├── array type: Array{Float32}
├── including: [:grid, :coriolis, :buoyancy, :closure]
└── max filesize: Inf YiB

In [ ]:
run!(simulation)